<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Servidores en Serie

In [29]:
import random
import math

In [30]:
# Parámetros
lmbda = 1/3 # tasa de llegadas
mu1 = 1/2 # tasa servicio S1
mu2 = 1/2 # tasa servicio S2
T_FIN = 50 # tiempo total de simulación

# Estado inicial
t = 0.0 # tiempo
NLL = 0 # número de llegadas
C1 = 0 # atendidos por S1
C2 = 0 # atendidos por S2

s1 = 0 # id del cliente en S1 (0 = libre)
s2 = 0 # id del cliente en S2 (0 = libre)
cola = [] # clientes en espera (ids)

# tiempos de eventos
t_LL = t + random.expovariate(lmbda)
t1 = math.inf
t2 = math.inf

LL = {}
S = {}

In [31]:
def servicio(mu):
    return random.expovariate(mu)

In [32]:
while t < T_FIN:

    if t_LL <= t1 and t_LL <= t2:
        evento = "llegada"
    elif t1 <= t2 and t1 < t_LL:
        evento = "fin1"
    else:
        evento = "fin2"

    # CASO 1: LLEGADA
    if evento == "llegada":
        t = t_LL
        NLL += 1
        cliente = NLL
        LL[cliente] = t

        # programar siguiente llegada
        t_LL = t + random.expovariate(lmbda)

        # asignación a servidor / cola
        if s1 == 0: # S1 libre
            s1 = cliente
            t1 = t + servicio(mu1)
        elif s2 == 0: # S2 libre
            s2 = cliente
            t2 = t + servicio(mu2)
        else: # ambos ocupados → a la cola
            cola.append(cliente)

    # CASO 2: FIN DE SERVICIO EN S1
    elif evento == "fin1":
        t = t1
        if s1 == 0:
            # en teoría no debería pasar
            t1 = math.inf
            continue

        cliente = s1
        C1 += 1
        S[cliente] = t # salida del sistema

        if cola:
            # pasa el siguiente de la cola a S1
            s1 = cola.pop(0)
            t1 = t + servicio(mu1)
        else:
            s1 = 0
            t1 = math.inf

    # CASO 3: FIN DE SERVICIO EN S2
    else: # evento == "fin2"
        t = t2
        if s2 == 0:
            t2 = math.inf
            continue

        cliente = s2
        C2 += 1
        S[cliente] = t

        if cola:
            s2 = cola.pop(0)
            t2 = t + servicio(mu2)
        else:
            s2 = 0
            t2 = math.inf

In [33]:
print("Tiempo final simulado:", t)
print("Llegadas totales :", NLL)
print("Servidos por S1 :", C1)
print("Servidos por S2 :", C2)

# tiempo medio en el sistema
clientes_salidos = S.keys()
if clientes_salidos:
    tiempos_sistema = [S[i] - LL[i] for i in clientes_salidos]
    print("Tiempo medio en el sistema:",
          sum(tiempos_sistema) / len(tiempos_sistema))
else:
    print("Nadie logró salir del sistema.")

Tiempo final simulado: 51.754425203317034
Llegadas totales : 16
Servidos por S1 : 9
Servidos por S2 : 6
Tiempo medio en el sistema: 1.4520200089421047
